In [1]:
using MLJ
MLJ.Random.seed!(1234)

┌ Info: Recompiling stale cache file /Users/anthony/.julia/compiled/v1.1/MLJ/rAU56.ji for MLJ [add582a8-e3ab-11e8-2d5e-e98b27df1bc7]
└ @ Base loading.jl:1184
┌ Info: Loading model metadata
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/MLJ/src/MLJ.jl:114


MersenneTwister(UInt32[0x000004d2])

## Getting some data:

In [2]:
using RDatasets
iris = dataset("datasets", "iris"); # a DataFrame
scrambled = shuffle(1:size(iris, 1))
X = iris[scrambled, 1:4];
y = iris[scrambled, 5];

first(X, 4)

┌ Info: Recompiling stale cache file /Users/anthony/.julia/compiled/v1.1/RDatasets/JyIbx.ji for RDatasets [ce6b1742-4840-55fa-b093-852dadbb1d8b]
└ @ Base loading.jl:1184


,SepalLength,SepalWidth,PetalLength,PetalWidth
,Float64,Float64,Float64,Float64
1,7.9,3.8,6.4,2.0
2,6.4,3.1,5.5,1.8
3,5.4,3.7,1.5,0.2
4,6.3,3.3,6.0,2.5


In [3]:
y[1:5]

5-element CategoricalArray{String,1,UInt8}:
 "virginica" 
 "virginica" 
 "setosa"    
 "virginica" 
 "versicolor"

## Basic fit and predict:

In [4]:
@load SVC()
classifier_ = SVC()
classifier = machine(classifier_, X, y)
fit!(classifier)
ŷ = predict(classifier, X) # or some Xnew

import MLJModels 

┌ Info: Recompiling stale cache file /Users/anthony/.julia/compiled/v1.1/MLJModels/0nkqq.ji for MLJModels [d491faf4-2d78-11e9-2867-c94bc002c0b7]
└ @ Base loading.jl:1184


✔
import LIBSVM 

┌ Info: Recompiling stale cache file /Users/anthony/.julia/compiled/v1.1/LIBSVM/3eWAI.ji for LIBSVM [b1bec4e5-fd48-53fe-b0cb-9723c09d164b]
└ @ Base loading.jl:1184


✔
import MLJModels.LIBSVM_.SVC ✔


┌ Info: Training Machine{SVC} @ 1…62.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/MLJ/src/machines.jl:140


*
optimization finished, #iter = 30
nu = 0.038907
obj = -1.945147, rho = -0.167873
nSV = 10, nBSV = 0
*
optimization finished, #iter = 40
nu = 0.293535
obj = -21.377490, rho = -0.144562
nSV = 33, nBSV = 26
*
optimization finished, #iter = 40
nu = 0.046518
obj = -2.403413, rho = 0.039689
nSV = 11, nBSV = 2
Total nSV = 45


150-element Array{CategoricalString{UInt8},1}:
 "virginica" 
 "virginica" 
 "setosa"    
 "virginica" 
 "versicolor"
 "setosa"    
 "setosa"    
 "virginica" 
 "virginica" 
 "setosa"    
 "setosa"    
 "versicolor"
 "versicolor"
 ⋮           
 "virginica" 
 "versicolor"
 "virginica" 
 "virginica" 
 "versicolor"
 "versicolor"
 "versicolor"
 "virginica" 
 "virginica" 
 "virginica" 
 "virginica" 
 "versicolor"

## Evaluating the model:

In [5]:
evaluate!(classifier,
          resampling=Holdout(fraction_train=0.8),
          measure=misclassification_rate)
# ## Adding dimension reduction:
@load PCA
dim_reducer_ = PCA()
dim_reducer = machine(dim_reducer_, X)
fit!(dim_reducer)
Xsmall = transform(dim_reducer, X);

first(Xsmall, 3)

┌ Info: Evaluating using a holdout set. 
│ fraction_train=0.8 
│ shuffle=false 
│ measure=MLJ.misclassification_rate 
│ operation=StatsBase.predict 
│ Resampling from all rows. 
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/MLJ/src/resampling.jl:100


import MLJModels ✔
import MultivariateStats 

┌ Info: Recompiling stale cache file /Users/anthony/.julia/compiled/v1.1/MultivariateStats/l7I74.ji for MultivariateStats [6f286f6a-111f-5878-ab1e-185364afe411]
└ @ Base loading.jl:1184


✔
import MLJModels.MultivariateStats_.PCA ✔


┌ Info: Training Machine{PCA} @ 2…49.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/MLJ/src/machines.jl:140


,x1,x2,x3
,Float64,Float64,Float64
1,-3.23067,1.37417,0.114548
2,-1.9051,0.0493005,-0.16218
3,2.50695,0.645069,0.075318


In [6]:
classifier = machine(classifier_, Xsmall, y)
fit!(classifier)
ŷ = predict(classifier, Xsmall)

*
optimization finished, #iter = 34
nu = 0.041934
obj = -2.096171, rho = -0.184206
nSV = 9, nBSV = 0
*
optimization finished, #iter = 54
nu = 0.271828
obj = -20.154548, rho = -0.053379
nSV = 32, nBSV = 24
*
optimization finished, #iter = 43
nu = 0.044759
obj = -2.302729, rho = 0.055344
nSV = 9, nBSV = 1
Total nSV = 42


┌ Info: Training Machine{SVC} @ 1…43.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/MLJ/src/machines.jl:140


150-element Array{CategoricalString{UInt8},1}:
 "virginica" 
 "virginica" 
 "setosa"    
 "virginica" 
 "versicolor"
 "setosa"    
 "setosa"    
 "virginica" 
 "virginica" 
 "setosa"    
 "setosa"    
 "versicolor"
 "versicolor"
 ⋮           
 "virginica" 
 "versicolor"
 "virginica" 
 "virginica" 
 "versicolor"
 "versicolor"
 "versicolor"
 "virginica" 
 "virginica" 
 "virginica" 
 "virginica" 
 "versicolor"

## Building a composite model:

### Method 1: Compact syntax (but not generalizable):

In [7]:
composite = @pipeline dim_reducer_ classifier_ # not implemented at time of talk

composite_ = machine(composite, X, y)
evaluate!(composite_, measure=misclassification_rate)

LoadError: UndefVarError: @pipeline not defined

### Method 2: Just write the math (and test as you build):

In [8]:
Xraw = X;
yraw = y;

X = source(Xraw)
y = source(yraw)

dim_reducer = machine(dim_reducer_, X)
Xsmall = transform(dim_reducer, X)

classifier = machine(classifier_, Xsmall, y)
ŷ = predict(classifier, Xsmall)

ŷ(rows=3:4)

fit!(ŷ)

ŷ(rows=3:4)

ErrorException: [34mNodalMachine{PCA} @ 9…70[39m has not been trained.

In [9]:
dim_reducer_.maxoutdim = 10
fit!(ŷ)

┌ Info: Training NodalMachine{PCA} @ 9…70.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/MLJ/src/machines.jl:140
┌ Info: Training NodalMachine{SVC} @ 1…69.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/MLJ/src/machines.jl:140


*
optimization finished, #iter = 34
nu = 0.041934
obj = -2.096171, rho = -0.184206
nSV = 9, nBSV = 0
*
optimization finished, #iter = 54
nu = 0.271828
obj = -20.154548, rho = -0.053379
nSV = 32, nBSV = 24
*
optimization finished, #iter = 43
nu = 0.044759
obj = -2.302729, rho = 0.055344
nSV = 9, nBSV = 1
Total nSV = 42


Node @ 1…12 = predict(1…69, transform(9…70, 5…42))

In [10]:
ŷ(rows=3:4)

2-element Array{CategoricalString{UInt8},1}:
 "setosa"   
 "virginica"

 Changing classifier hyperparameter does not retrigger retraining of
 upstream dimension reducer:

In [11]:
classifier_.gamma = 0.1
fit!(ŷ)

*
optimization finished, #iter = 33
nu = 0.035524
obj = -1.854426, rho = -0.114264
nSV = 7, nBSV = 1
*
optimization finished, #iter = 35
nu = 0.403719
obj = -29.357598, rho = -0.027988
nSV = 42, nBSV = 39
*
optimization finished, #iter = 15
nu = 0.066672
obj = -3.605309, rho = -0.043427
nSV = 9, nBSV = 5
Total nSV = 53


┌ Info: Not retraining NodalMachine{PCA} @ 9…70.
│  It appears up-to-date. Use force=true to force retraining.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/MLJ/src/machines.jl:146
┌ Info: Updating NodalMachine{SVC} @ 1…69.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/MLJ/src/machines.jl:152


Node @ 1…12 = predict(1…69, transform(9…70, 5…42))

In [12]:
ŷ(rows=3:4)

2-element Array{CategoricalString{UInt8},1}:
 "setosa"   
 "virginica"

#### Exporting network as stand-alone re-usable model:

In [13]:
composite = @from_network Composite(pca=dim_reducer_, svc=classifier_) <= (X, y, ŷ)
params(composite)

(pca = (maxoutdim = 10,
        method = :auto,
        pratio = 0.99,
        mean = nothing,),
 svc = (kernel = RadialBasis::KERNEL = 2,
        gamma = 0.1,
        weights = nothing,
        cost = 1.0,
        degree = 3,
        coef0 = 0.0,
        tolerance = 0.001,
        shrinking = true,
        probability = false,),)

In [14]:
composite_ = machine(composite, Xraw, yraw)
evaluate!(composite_, measure=misclassification_rate)

┌ Info: Evaluating using cross-validation. 
│ nfolds=6. 
│ shuffle=false 
│ measure=MLJ.misclassification_rate 
│ operation=StatsBase.predict 
│ Resampling from all rows. 
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/MLJ/src/resampling.jl:151
Cross-validating: 100%[=========================] Time: 0:00:02


6-element Array{Float64,1}:
 0.0 
 0.0 
 0.04
 0.0 
 0.08
 0.12

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*